# TESTE PRÁTICO DE LINGUAGEM DE CONSULTA ESTRUTURADA (SQL)
Para a realização das atividades, você deverá executar as instruções abaixo para importar as bibliotecas e funções necessárias para criar a base de dados.

Ao final, esperamos que você desenvolva os códigos selecionados e entregue o notebook com as consultas SQL desenvolvidas e o arquivo .db criado.

O teste importará três arquivos cruciais para o desenvolvimento dessa atividade. Todos estão no Githun (não mude as URL's).

*   usuarios.csv
*   produtos.csv
*   vendas.csv

## O teste foi desenvolvido para ser executado com o SQLite3 e no Google Colab (Ubuntu 18.04). Você deverá utilizar consultas no padrão ANSI. Os comandos necessários podem ser encontrados em:

https://www.sqlite.org/docs.html

https://www.tutorialspoint.com/sqlite/index.htm



# AVISO!



1.   Execute o código abaixo para atualizar a biblioteca do SQLite3 utilizada para essa atividade
2.   **Ignore as mensagens de erro sobre perda de conectividade na instrução abaixo (o kernel será reiniciado para atualização do SQLite3)**


In [ ]:
import os

# BAIXAR A ÚLTIMA RELEASE DO COLAB, DESCOMPACTAR E INSTALAR NO DIRETÓRIO DO 
# PYTHON 3.7 (VERSÃO USADA PELO COLAB NESSE NOTEBOOK)
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content/sqlite/
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

# REINICIANDO O KERNEL DO COLAB PARA IMPORTAR A BIBLIOTECA ATUALIZADA
os.kill(os.getpid(), 9)

# INSTRUÇÕES PARA REALIZAR AS CONSULTAS
### PARA CONSULTAR DADOS, SIGA OS PASSOS ABAIXO:

#### Abrir conexão:
    conexao_sql = sqlite3.connect(banco)
#### Abrir cursor para executar consulta no banco:
    cursor = conexao_sql.cursor()
#### Criar uma consulta:
    consulta = '''
        Coloque aqui a sua instrução SQL;
    '''
    cursor.execute(consulta)
#### Para visualizar o resultado da sua consulta:
    for linha in cursor:
      print(linha)
#### Para fechar a conexão:
    conexao_sql.close()

### LEMBRE-SE DE QUE PYTHON É SENSÍVEL A IDENTAÇÃO DE CÓDIGO!

# IMPORTANDO BIBLIOTECAS NECESSÁRIAS PARA EXECUTAR OS CÓDIGOS EM SQL

In [4]:
import sqlite3
import csv
import requests
from pathlib import Path
from urllib.parse import urlparse

# DEFINIÇÃO DE VARIÁVEIS FIXAS PARA O PROJETO

In [5]:
banco = "teste_sql.db"

# FUNÇÃO PARA INSERIR REGISTROS NO BANCO DE DADOS

In [6]:
# Recebe como parâmetro
# 1. Nome do banco de dados
# 2. Nome da tabela que receberá os dados
# 3. Lista das colunas da tabela
# 4. Dados que serão inseridos

def inserir_registros(banco, tabela, lista_colunas, arquivo):
  conexao_sql = sqlite3.connect(banco)
  cursor = conexao_sql.cursor()
  dados = open(arquivo)
  conteudo_dados = csv.reader(dados)
  num_colunas = 1 if lista_colunas.count(',') == 1 else lista_colunas.count(',')
  string_insert = 'INSERT INTO ' + tabela + ' (' + lista_colunas + ') VALUES (?' + ('' if num_colunas == 1 else ', ?' * (num_colunas)) + ')'
  cursor.executemany(string_insert, conteudo_dados)
  conexao_sql.commit()
  conexao_sql.close()

# FUNÇÃO PARA BAIXAR A BASE DE DADOS UTILIZADA PARA DESENVOLVIMENTO DAS ATIVIDADES

In [7]:
# Recebe como parâmetro:
# 1. Nome do arquivo com a extensão (Ex: base.csv)
# 2. URL do site onde o dado está localizado (Ex: 'https://drive.google.com/user_teste/folder_dataset/file.csv'):

def baixar_dados(nome_arquivo, url):
  requisicao = requests.get(url)
  conteudo = requisicao.content
  arquivo_csv = open(nome_arquivo, 'wb')
  arquivo_csv.write(conteudo)
  arquivo_csv.close()

# CRIAÇÃO DE CONEXÃO COM O BANCO SQLITE3 PARA DESENVOLVIMENTO DA ATIVIDADE

In [8]:
# Abre conexão com o banco. Se o arquivo não existir, cria a base.
conexao_sql = sqlite3.connect(banco)
conexao_sql.close()

# CRIAÇÃO DAS TABELAS PARA REALIZAÇÃO DAS ATIVIDADES

### TABELA DE USUÁRIOS

In [9]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/usuarios.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS usuarios
(
  cod_usuario INTEGER NOT NULL,
  data_cadastro DATE NOT NULL,
  faixa_etaria TEXT,
  cidade TEXT NOT NULL,
  estado TEXT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, data_cadastro, faixa_etaria, cidade, estado'
inserir_registros(banco, 'usuarios', lista_campos, nome_arquivo)

### TABELA DE PRODUTOS

In [10]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/produtos.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_produtos = 'produtos.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS produtos
(
  cod_produto INTEGER NOT NULL,
  nome_produto TEXT NOT NULL,
  categoria_produto TEXT,
  valor_produto FLOAT);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_produto, nome_produto, categoria_produto, valor_produto'
inserir_registros(banco, 'produtos', lista_campos, arquivo_produtos)

### TABELA DE VENDAS

In [11]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/vendas.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_vendas = 'vendas.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS vendas
(
  cod_usuario INTEGER NOT NULL,
  cod_produto INTEGER NOT NULL,
  data_compra DATE NOT NULL,
  quantidade INT NOT NULL,
  valor FLOAT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, cod_produto, data_compra, quantidade, valor'
inserir_registros(banco, 'vendas', lista_campos, arquivo_vendas)

# PERGUNTA 1:

### Escreva um comando em SQL que retorne a quantidade de vendas dos estados de Minas Gerais e São Paulo por ano e mês

In [36]:
#
# Criando uma função para executar o cursor via context manager
#
# (esta importação, claro, estaria no início)
from contextlib import contextmanager

@contextmanager
def execute_query(query):
    with sqlite3.connect(banco) as conexao_sql:
        c = conexao_sql.cursor()
        c.execute(query)
        yield c
        c.close()

In [44]:
query = '''
    SELECT u.estado,
           strftime('%Y-%m', v.data_compra),
           sum(quantidade) qtd_vendas
      FROM vendas v
      JOIN usuarios u ON
        u.cod_usuario = v.cod_usuario
      WHERE u.estado IN ('São Paulo', 'Minas Gerais')
      GROUP BY u.estado, strftime('%Y-%m', v.data_compra)
'''

with execute_query(query) as rows:
  for row in rows:
    print(row)


('Minas Gerais', '2018-06', 10)
('Minas Gerais', '2018-08', 16)
('Minas Gerais', '2018-10', 20)
('Minas Gerais', '2018-11', 10)
('Minas Gerais', '2018-12', 39)
('Minas Gerais', '2019-01', 20)
('Minas Gerais', '2019-02', 2)
('Minas Gerais', '2019-04', 26)
('Minas Gerais', '2019-05', 11)
('Minas Gerais', '2019-06', 1)
('Minas Gerais', '2019-07', 14)
('Minas Gerais', '2019-08', 15)
('Minas Gerais', '2019-09', 27)
('Minas Gerais', '2019-10', 3)
('Minas Gerais', '2019-11', 1)
('Minas Gerais', '2019-12', 25)
('Minas Gerais', '2020-01', 8)
('Minas Gerais', '2020-02', 10)
('Minas Gerais', '2020-03', 14)
('Minas Gerais', '2020-04', 1)
('Minas Gerais', '2020-05', 44)
('Minas Gerais', '2020-06', 18)
('Minas Gerais', '2020-07', 30)
('Minas Gerais', '2020-09', 44)
('Minas Gerais', '2020-10', 8)
('Minas Gerais', '2020-11', 43)
('Minas Gerais', '2020-12', 18)
('Minas Gerais', '2021-01', 21)
('Minas Gerais', '2021-02', 75)
('Minas Gerais', '2021-03', 6)
('Minas Gerais', '2021-04', 58)
('Minas Gerais',

# PERGUNTA 2:

### Escreva um comando em SQL que retorne o ticket médio (valor médio de compra) e quantidade de compras realizadas dos usuários que fizeram compras entre 2019 e 2020 e que tiveram mais de 3 compras no período (lembre-se de contar todas as compras dos usuários que estão nessa condição!)

In [63]:
query = '''
    SELECT u.cod_usuario,
           ROUND(AVG(valor),2) avg_compra, 
           count(1) qtd_compras
      FROM vendas v
      JOIN usuarios u
        ON u.cod_usuario = v.cod_usuario
      WHERE v.data_compra BETWEEN '2019-01-01' AND '2020-12-31'
      GROUP BY u.cod_usuario
      HAVING COUNT(1) > 3
'''

with execute_query(query) as rows:
  for row in rows:
    print(row)

(69, 25926.09, 4)
(89, 22791.03, 4)
(176, 1686.69, 5)
(191, 1971.87, 4)
(441, 2147.87, 6)
(700, 21861.0, 6)
(796, 1696.88, 4)
(898, 3710.48, 4)
(1074, 21402.8, 7)
(1192, 2565.63, 4)
(1415, 210.56, 6)
(1696, 252.88, 6)
(1752, 13832.69, 7)
(1756, 24269.69, 5)
(1787, 15756.61, 7)
(1947, 14866.73, 4)
(2183, 1482.16, 5)
(2340, 22284.58, 6)
(2392, 8185.07, 4)
(2438, 17550.01, 5)
(2551, 17265.39, 6)
(2597, 8681.9, 4)
(2646, 12171.34, 5)
(2669, 6578.05, 4)
(2811, 20773.65, 4)
(2843, 16535.01, 8)
(2856, 13890.9, 5)
(2920, 15125.48, 5)
(2981, 21811.9, 5)
(3090, 13108.06, 7)
(3358, 27187.44, 5)
(3403, 14547.95, 8)


# PERGUNTA 3:

### Escreva um comando em SQL que retorne quais são os usuários por estado que mais compraram em todo o período analisado e, o número de compras realizadas, a quantidade total de itens comprados e valor total pago

In [86]:
# limitando aos 5 maiores compradores de cada estado.
# alterar o limite, caso necessário.
query = '''
  SELECT estado,
         cod_usuario,
         sum_compras,
         qtd_compras
    FROM (
      SELECT u.estado,
            u.cod_usuario,
            ROUND(SUM(valor),2) sum_compras,
            count(1) qtd_compras,
            row_number() over (partition by u.estado order by count(1) DESC) as rank    
        FROM vendas v
        JOIN usuarios u
          ON u.cod_usuario = v.cod_usuario
      GROUP BY u.estado, u.cod_usuario
    ) AS rank
  WHERE rank <= 5
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

('Amapá', 2186, 51359.34, 7)
('Bahia', 2981, 126282.4, 9)
('Bahia', 2856, 71986.05, 7)
('Bahia', 3400, 7052.29, 5)
('Bahia', 176, 8433.46, 5)
('Bahia', 2155, 13820.4, 4)
('Ceará', 1787, 121494.67, 8)
('Goiás', 700, 142389.09, 9)
('Goiás', 3358, 136881.19, 7)
('Goiás', 2850, 107155.5, 5)
('Goiás', 1878, 96862.1, 4)
('Maranhão', 2811, 184101.89, 13)
('Maranhão', 441, 14945.13, 8)
('Maranhão', 191, 26216.35, 7)
('Mato Grosso', 2340, 165197.3, 9)
('Mato Grosso', 1192, 16001.5, 7)
('Mato Grosso do Sul', 679, 152271.96, 6)
('Mato Grosso do Sul', 2492, 17998.0, 3)
('Minas Gerais', 3090, 95697.3, 9)
('Minas Gerais', 2438, 95031.67, 8)
('Minas Gerais', 2216, 85841.41, 7)
('Minas Gerais', 796, 28161.0, 7)
('Minas Gerais', 2009, 24173.87, 6)
('Paraná', 2843, 137228.01, 10)
('Paraná', 2597, 55295.79, 6)
('Paraná', 2392, 41412.09, 6)
('Paraná', 898, 15390.8, 5)
('Paraíba', 464, 73556.1, 6)
('Paraíba', 1796, 54689.7, 3)
('Pernambuco', 2669, 37119.0, 7)
('Pernambuco', 2646, 135556.7, 7)
('Piauí', 244

# PERGUNTA 4:

### Escreva um comando em SQL que retorne quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga

In [105]:
query = '''
   SELECT estado,
          cod_produto,
          qtd_itens,
          sum_compras,
          avg_compras
     FROM (
      SELECT u.estado,
            p.cod_produto, 
            count(1) qtd_itens,
            ROUND(SUM(valor),2) sum_compras,
            ROUND(AVG(valor),2) avg_compras,
            row_number() over (partition by u.estado order by count(1) DESC) as rank
        FROM vendas v
        JOIN usuarios u
          ON u.cod_usuario = v.cod_usuario
        JOIN produtos p
          ON p.cod_produto = v.cod_produto
        WHERE u.estado in ("Paraná", "Santa Catarina", "Rio Grande do Sul",
                           "Espírito Santo", "Minas Gerais", "Rio de Janeiro","São Paulo")
      GROUP BY p.cod_produto
     ) AS rank
   WHERE rank <= 3
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

('Minas Gerais', 15, 26, 1397.2, 53.74)
('Minas Gerais', 1, 14, 615180.6, 43941.47)
('Minas Gerais', 5, 13, 61617.0, 4739.77)
('Paraná', 2, 6, 95550.38, 15925.06)
('Rio Grande do Sul', 7, 3, 67625.48, 22541.83)
('São Paulo', 17, 14, 56073.6, 4005.26)
('São Paulo', 10, 11, 81188.4, 7380.76)
('São Paulo', 16, 8, 270.0, 33.75)


# PERGUNTA 5:

### Escreva um comando em SQL que retorne o ticket médio e o número total de usuários que fizeram pelo menos uma compra por faixa etária

In [112]:
query = '''
    SELECT u.faixa_etaria,
           ROUND(AVG(v.valor),2) tkt_medio,
           COUNT(1) qtd_usuarios,
      FROM vendas v
      JOIN usuarios u
        ON u.cod_usuario = v.cod_usuario
    GROUP BY u.faixa_etaria
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

('Entre 10 a 15 anos', 8890.13, 43)
('Entre 16 a 21 anos', 12725.95, 50)
('Entre 22 a 27 anos', 12203.02, 64)
('Entre 28 a 36 anos', 8233.04, 56)
('Entre 37 a 49 anos', 16727.19, 42)
('Entre 50 a 61 anos', 10849.48, 80)
('Entre 62 a 70 anos', 10012.63, 39)
('Mais de 70 anos', 11303.1, 29)


# PERGUNTA 6:

### Escreva um comando em SQL que aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado

In [126]:
query = '''
    SELECT cod_usuario,
        cod_produto,
        valor,
        ROUND((valor - (valor * 0.10)), 2) valor_com_desc
    FROM (
        SELECT u.cod_usuario,
            v.cod_produto,
            v.valor,
            row_number() over (partition by u.cod_usuario, p.categoria_produto) as rank  
        FROM vendas v
        JOIN usuarios u
            ON u.cod_usuario = v.cod_usuario
        JOIN produtos p
          ON p.cod_produto = v.cod_produto
        ) as q
    WHERE rank > 3
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

(2438, 15, 39.92, 35.93)
(2811, 1, 36727.2, 33054.48)
(2811, 1, 18363.6, 16527.24)


# PERGUNTA 7

### Escreva um comando em SQL que retorna o produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago

In [135]:
query = '''
    SELECT v.cod_produto,
           count(1) qtd_usuario,
           min(v.valor) min_valor,
           max(v.valor) max_valor
      FROM vendas v
      JOIN usuarios u
        ON u.cod_usuario = v.cod_usuario
    GROUP BY
      v.cod_produto
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

(1, 29, 4590.9, 82636.2)
(2, 15, 2691.56, 26915.6)
(3, 12, 4099.0, 40990.0)
(4, 12, 58.9, 1178.0)
(5, 27, 874.0, 8740.0)
(6, 21, 899.9, 17098.1)
(7, 13, 1300.49, 24709.31)
(8, 21, 219.57, 3732.69)
(9, 13, 389.1, 6614.7)
(10, 19, 2099.7, 13298.1)
(11, 18, 49.9, 848.3)
(12, 21, 77.5, 1550.0)
(13, 16, 29.7, 198.0)
(14, 18, 138.99, 2779.8)
(15, 36, 4.99, 99.8)
(16, 18, 5.0, 100.0)
(17, 28, 584.1, 11097.9)
(18, 13, 7.9, 158.0)
(19, 13, 2499.9, 42498.3)
(20, 13, 129.9, 2598.0)
(21, 27, 5999.1, 119982.0)


# PERGUNTA 8

### Escreva um comando em SQL que retorne o total de compras realizadas, o total de itens comprados e a receita total obtida

In [134]:
query = '''
    SELECT count(1) tot_compras,
           ROUND(sum(quantidade),2) sum_items_vendidos,
           ROUND(sum(valor),2) tot_receita
      FROM vendas
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

(20000, 209149.0, 215849001.66)


# PERGUNTA 9

### Escreva um comando em SQL que retorne o produto, a quantidade de itens vendidos e a receita do produto que tem mais itens vendidos por ano e mês

In [164]:
query = '''
  SELECT ano_mes,
         cod_produto,
         sum_itens_vendidos,
         tot_receita
    FROM (
      SELECT strftime('%Y-%m', data_compra) ano_mes,
            cod_produto,
            sum(quantidade) sum_itens_vendidos,
            ROUND(sum(valor),2) tot_receita,
            row_number() over (partition by strftime('%Y-%m', data_compra) order by sum(quantidade) DESC) as rank
        FROM vendas
      GROUP BY
        strftime('%Y-%m', data_compra),
        cod_produto
      ) AS Q
    WHERE rank = 1
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

('2018-01', 3, 17, 34841.5)
('2018-02', 11, 35, 1746.5)
('2018-03', 16, 38, 190.0)
('2018-04', 3, 50, 102475.0)
('2018-05', 5, 43, 18791.0)
('2018-06', 12, 47, 3642.5)
('2018-07', 10, 62, 43393.8)
('2018-08', 2, 89, 119774.42)
('2018-09', 7, 87, 113142.63)
('2018-10', 17, 99, 57825.9)
('2018-11', 1, 85, 390226.5)
('2018-12', 18, 102, 805.8)
('2019-01', 21, 135, 809878.5)
('2019-02', 9, 82, 31906.2)
('2019-03', 4, 142, 8363.8)
('2019-04', 15, 167, 833.33)
('2019-05', 2, 164, 220707.92)
('2019-06', 11, 177, 8832.3)
('2019-07', 19, 234, 584976.6)
('2019-08', 3, 273, 559513.5)
('2019-09', 1, 210, 964089.0)
('2019-10', 20, 222, 28837.8)
('2019-11', 5, 252, 110124.0)
('2019-12', 4, 321, 18906.9)
('2020-01', 8, 301, 66090.57)
('2020-02', 7, 282, 366738.18)
('2020-03', 11, 329, 16417.1)
('2020-04', 3, 343, 702978.5)
('2020-05', 7, 395, 513693.55)
('2020-06', 10, 416, 291158.4)
('2020-07', 16, 395, 1975.0)
('2020-08', 11, 461, 23003.9)
('2020-09', 6, 446, 401355.4)
('2020-10', 13, 482, 4771.8)


# PERGUNTA 10

### Escreva um comando em SQL que exiba quem são os usuários que compraram produtos diferentes

In [169]:
# irei considerar que os produtos distintos foram comprados
# na mesma data (possivelmente mesma compra?)
query = '''
    SELECT DISTINCT v1.cod_usuario
    FROM vendas v1
    JOIN vendas v2
      ON v1.cod_usuario = v2.cod_usuario
      AND v1.data_compra = v2.data_compra
      AND v1.cod_produto <> v2.cod_produto
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

(1919,)
(3272,)
(2514,)
(314,)
(3389,)
(881,)
(453,)
(104,)
(2,)
(1091,)
(1501,)
(285,)
(2722,)
(107,)
(3119,)
(1118,)
(2977,)
(3160,)
(2822,)
(136,)
(3408,)
(2624,)
(2260,)
(840,)
(1923,)
(997,)
(2349,)
(2909,)
(1491,)
(3300,)
(719,)
(1139,)
(642,)
(1051,)
(778,)
(320,)
(843,)
(674,)
(53,)
(395,)
(979,)
(2573,)
(2416,)
(898,)
(761,)
(1210,)
(2223,)
(1449,)
(132,)
(31,)
(1495,)
(2469,)
(1251,)
(437,)
(2013,)
(17,)
(1398,)
(119,)
(1525,)
(1310,)
(1691,)
(1967,)
(3109,)
(2778,)
(2950,)
(1023,)
(3156,)
(2975,)
(1330,)
(277,)
(1178,)
(1855,)
(684,)
(431,)
(1911,)
(211,)
(1942,)
(3317,)
(2355,)
(1781,)
(2805,)
(189,)
(2356,)
(2627,)
(788,)
(421,)
(50,)
(2188,)
(1022,)
(1766,)
(516,)
(309,)
(452,)
(1058,)
(2640,)
(2633,)
(1106,)
(2197,)
(621,)
(1422,)
(1142,)
(411,)
(2396,)
(2581,)
(2400,)
(333,)
(71,)
(404,)
(3333,)
(968,)
(1298,)
(2999,)
(1917,)
(8,)
(533,)
(2199,)
(2130,)
(1521,)
(2985,)
(15,)
(2852,)
(1465,)
(635,)
(928,)
(1936,)
(1131,)
(2745,)
(68,)
(3339,)
(3,)
(715,)
(1924,)
(2027,)


# PERGUNTA 11

### Escreva um comando em SQL que verifique usuários que não fizeram nenhuma compra

In [175]:
query = '''
    SELECT u.cod_usuario
    FROM usuarios u
    LEFT JOIN vendas v
      ON v.cod_usuario = u.cod_usuario
    WHERE v.cod_usuario IS NULL
'''

with execute_query(query) as rows:
    for row in rows:
        print(row)

('cod_usuario',)
(5098,)
(5229,)
(5482,)
(5504,)
(5614,)
(5923,)
(6985,)
(7901,)
(9980,)
(10080,)
(10152,)
(10170,)
(10929,)
(11002,)
(11546,)
(12298,)
(12354,)
(13117,)
(13476,)
(14486,)
(15198,)
(15313,)
(15784,)
(16570,)
(17530,)
(17779,)
(18028,)
(18034,)
(18166,)
(18366,)
(19078,)
(19231,)
(19445,)
(19590,)
(19798,)
(19984,)
(20067,)
(20207,)
(20425,)
(21347,)
(21440,)
(21481,)
(22113,)
(22188,)
(22290,)
(22321,)
(22470,)
(22692,)
(23077,)
(23127,)
(23618,)
(23732,)
(23858,)
(24216,)
(24470,)
(24951,)
(25517,)
(25638,)
(25923,)
(26156,)
(26514,)
(27003,)
(27222,)
(27417,)
(27642,)
(27859,)
(27870,)
(28538,)
(28877,)
(29003,)
(29581,)
(29611,)
(30117,)
(30362,)
(30697,)
(31115,)
(31286,)
(31765,)
(32872,)
(32932,)
(33574,)
(33661,)
(33684,)
(33869,)
(34126,)
(34180,)
(34232,)
(34324,)
(35041,)
(35167,)
(35524,)
(35656,)
(35700,)
(35855,)
(36314,)
(36354,)
(36548,)
(36738,)
(36742,)
(36935,)
(37127,)
(37331,)
(38002,)
(38248,)
(38779,)
(38819,)
(39239,)
(39388,)
(39635,)
(40009,)
(4